# Setup : import + get data

## Import

### Import classiques

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import ipdb

### Import pour accès données (sql, google drive...)

In [ ]:
import pymysql
import sqlalchemy
from google.colab import drive

## Get data from data.gouv.fr

In [5]:
path_df2020 = 'https://files.data.gouv.fr/geo-dvf/latest/csv/2020/full.csv.gz' #data France entière année 2020
path_df2019 = 'https://files.data.gouv.fr/geo-dvf/latest/csv/2019/full.csv.gz' #data France entière année 2019
path_df2018 = 'https://files.data.gouv.fr/geo-dvf/latest/csv/2018/full.csv.gz' #data France entière année 2018
path_df2017 = 'https://files.data.gouv.fr/geo-dvf/latest/csv/2017/full.csv.gz' #data France entière année 2017
path_df2016 = 'https://files.data.gouv.fr/geo-dvf/latest/csv/2016/full.csv.gz' #data France entière année 2016

In [8]:
df2019 = pd.read_csv(path_df2019)
df2019.head()

/home/jean/.pyenv/versions/3.8.6/envs/reestimator/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10,12,14,16,17,18,20,22,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2019-1,2019-01-04,1,Vente,37220.0,26.0,NaN,RUE DE MONTHOLON,2730,1000.0,...,Appartement,20.0,1.0,NaN,NaN,NaN,NaN,NaN,5.209578,46.198844
1,2019-2,2019-01-04,1,Vente,185100.0,22.0,B,RUE GEN DELESTRAINT,1650,1000.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,5.219438,46.198784
2,2019-2,2019-01-04,1,Vente,185100.0,22.0,NaN,RUE GEN DELESTRAINT,1650,1000.0,...,Appartement,62.0,3.0,NaN,NaN,NaN,NaN,NaN,5.219438,46.198784
3,2019-3,2019-01-08,1,Vente,209000.0,3.0,NaN,RUE DES CHAMPAGNES,0043,1160.0,...,Maison,90.0,4.0,S,sols,NaN,NaN,940.0,5.282085,45.999533
4,2019-4,2019-01-07,1,Vente,134900.0,5.0,NaN,LOT LE BIOLAY,A003,1370.0,...,Maison,101.0,5.0,S,sols,NaN,NaN,490.0,5.299919,46.293275


## Get data from Google Drive (sur Google colab)

In [ ]:
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/reestimator-raw_data/Samples/'
file500 = 'sample500.csv' #échangillonage sur 500 000 lignes des données sur toute la france et les 5 années
file300 = 'sample300.csv' # idem 300 000 lignes
file200b = 'sample200b.csv'
file200 = 'sample200.csv'

In [ ]:
df500 = pd.read_csv(data_path+file500)
df500.head()

## Get data from database /!\ (bouffe du crédit GCP)

In [ ]:
class Data_loading:
    """
    Load Libraries and Create engine Connection
    """
    def __init__(self,
                 username='Estimators',
                 password='Estimators2021',
                 host='34.77.88.127',
                 port=3306,
                 database='Housing_France'):
        self.conn = self.establish_conn(username, password, host, port, database)


    def establish_conn(self,username, password, host, port, database):
        engine = sqlalchemy.create_engine(
                sqlalchemy.engine.url.URL.create(
                drivername="mysql+pymysql",
                username='Estimators',  # e.g. "my-database-user"1777777777
                password='Estimator2021',  # e.g. "my-database-password"
                host='34.77.88.127',  # e.g. "127.0.0.1"
                port=3306,  # e.g. 3306
                database='Housing_France',  # e.g. "my-database-name"
            ))
        return engine

    def load_data_chunk(self, table_name, chunksize):

        frame = pd.DataFrame()
        for chunk_dataframe in pd.read_sql(f"""Select * from {table_name}""",
                                           self.conn,
                                           chunksize=chunksize):
            print(f"Got dataframe w/{len(chunk_dataframe)} rows")
            frame = frame.append(chunk_dataframe)

        return frame


    def get_random_rows(self, table_name, numrows):

        df = pd.read_sql(
            f"""SELECT * FROM {table_name} dm ORDER BY RAND() LIMIT {numrows};""",
            self.conn)
        return df

    def get_all_rows(self, table_name):
        df = pd.read_sql(f"""SELECT * FROM {table_name} """, self.conn)
        return df

    def get_num_rows(self,table_name, rownums):
        df = pd.read_sql(f"""SELECT * FROM {table_name} Limit {rownums} """,
                         self.conn)

        return df

    def show_tables(self):
        print(self.conn)
        df = pd.read_sql(
        """SELECT TABLE_NAME FROM information_schema.tables where TABLE_SCHEMA = 'Housing_France'""",
            self.conn)
        return df

    def data_to_sql(self, df, tablename, if_exists):
        """Export Data to Sql, if exists takes one of the two strings :  ['replace','append'] """
        df.to_sql(name=f'{str(tablename)}',
                  con=self.conn,
                  if_exists=f'{if_exists}',
                  index=True)
        return print(f"the table {str(tablename)} was successfully loaded to DB")

    def get_data(self, querystring):
        df = pd.read_sql(querystring, self.conn)
        return df

In [ ]:
    dataloader = Data_loading()
    df_db = dataloader.load_data_chunk('data_working_update', 100000)

**Pour toute la suite nous travaillerons sur df500, et parfois sur une année en particulier (df2019)**
_pour que ça fonctionne sur jnotebook en local, on charge df500 depuis une copie dans raw_data_

In [9]:
df500 = pd.read_csv('../raw_data/sample500.csv')

/home/jean/.pyenv/versions/3.8.6/envs/reestimator/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Exploration

## Caractéristiques du dataframe : colonnes, dtypes, valeurs max...

### carac df500

In [10]:
df500.shape

(500000, 40)

In [12]:
df500.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [11]:
df500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 40 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id_mutation                   500000 non-null  object 
 1   date_mutation                 500000 non-null  object 
 2   numero_disposition            500000 non-null  int64  
 3   nature_mutation               500000 non-null  object 
 4   valeur_fonciere               500000 non-null  int64  
 5   adresse_numero                500000 non-null  int64  
 6   adresse_suffixe               36023 non-null   object 
 7   adresse_nom_voie              499991 non-null  object 
 8   adresse_code_voie             500000 non-null  object 
 9   code_postal                   500000 non-null  int64  
 10  code_commune                  500000 non-null  int64  
 11  nom_commune                   500000 non-null  object 
 12  code_departement              500000 non-nul

On constate qu'il y a certaines colonnes qui contiennent beaucoup de valeurs nulles. On a un problème d'homogénéité des données dans certaines colonnes (mélange de str et de valeurs numériques). Le type de données n'est pas toujours optimum (int64 pour numéro de voie, float64 pour code postal...)
Certaines colonnes n'ont pas vraiment d'utilité (correspondance ancien codage, etc.)

#### nbre de vente en fonction des natures de mutation

In [14]:
df500['nature_mutation'].unique()

array(['Vente', "Vente en l'état futur d'achèvement", 'Echange',
       'Adjudication', 'Expropriation', 'Vente terrain à bâtir'],
      dtype=object)

In [16]:
df500.groupby('nature_mutation')['id_mutation'].count()

nature_mutation
Adjudication                            2237
Echange                                 1321
Expropriation                            676
Vente                                 471031
Vente en l'état futur d'achèvement     24382
Vente terrain à bâtir                    353
Name: id_mutation, dtype: int64

Les ventes constituent l'essentiel des mutations. Tant mieux, c'est ce qui nous intéresse. On va ne sélectionner que les ventes.

#### Types de biens

In [17]:
df500['type_local'].unique()

array(['Dépendance', 'Maison', 'Appartement',
       'Local industriel. commercial ou assimilé'], dtype=object)

In [18]:
df500.groupby('type_local')['id_mutation'].count()

type_local
Appartement                                 160315
Dépendance                                  121348
Local industriel. commercial ou assimilé     34309
Maison                                      184028
Name: id_mutation, dtype: int64

Les ventes de maisons et d'appartement représentent la majorité des ventes. On constate que le nombre de dépendances est très important. Néanmoins, une dépendance n'est par définition vendue seule (ligne 

In [21]:
((df500.groupby('id_mutation')['id_mutation'].count() > 1) == True).count()

453992

In [22]:
((df2019.groupby('id_mutation')['id_mutation'].count() > 1) == True).count()

1514872

In [23]:
df2019.shape

(3533211, 40)

# Cleaning